# Hirschsprung Disease Classification CNN

## Setup and Imports

In [1]:
import glob
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

## Helper Functions

In [2]:
def load_and_preprocess_image(img_path, img_size=(224, 224)):
    """
    Load a grayscale image, resize, convert to 3-channel, and normalize.
    """
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise IOError(f"Cannot read image: {img_path}")
    img = cv2.resize(img, img_size)
    img = np.stack([img]*3, axis=-1).astype('float32') / 255.0
    return np.expand_dims(img, axis=0)

In [3]:
def build_model(input_shape=(224,224,3)):
    """
    Create DenseNet121 base with custom top for binary classification.
    """
    base = DenseNet121(weights=None, include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base.output)
    x = Dropout(0.25)(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(base.input, out)
    for layer in base.layers:
        layer.trainable = False
    model.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    return model

## Load Model

In [5]:
model = build_model()
model.load_weights('model/hd_classifier.h5')
print("Model loaded successfully.")

Model loaded successfully.


## Run Predictions

### Positive class example

In [6]:
image_dir = 'data/positive'
threshold = 0.46859
paths = glob.glob(f"{image_dir}/*")
results = []

print(f"Predicting {len(paths)} images from '{image_dir}'...\n")
for p in paths:
    img = load_and_preprocess_image(p)
    prob = float(model.predict(img, verbose=0)[0,0])
    label = int(prob > threshold)
    results.append(label)
    print(p, '| class =', label, f'| prob = {prob:.4f}')

if results:
    ratio = sum(results)/len(results)
    overall = int(ratio >= 0.5)
    print(f"\nOverall patient class = {overall} (ratio = {ratio:.2f})")
else:
    print("No images found.")

Predicting 3 images from 'data/positive'...

data/positive/Patient_1_Image_3.png | class = 1 | prob = 0.6942
data/positive/Patient_1_Image_2.png | class = 1 | prob = 0.7458
data/positive/Patient_1_Image_1.png | class = 1 | prob = 0.5016

Overall patient class = 1 (ratio = 1.00)


### Negative class example

In [7]:
image_dir = 'data/negative'
threshold = 0.46859
paths = glob.glob(f"{image_dir}/*")
results = []

print(f"Predicting {len(paths)} images from '{image_dir}'...\n")
for p in paths:
    img = load_and_preprocess_image(p)
    prob = float(model.predict(img, verbose=0)[0,0])
    label = int(prob > threshold)
    results.append(label)
    print(p, '| class =', label, f'| prob = {prob:.4f}')

if results:
    ratio = sum(results)/len(results)
    overall = int(ratio >= 0.5)
    print(f"\nOverall patient class = {overall} (ratio = {ratio:.2f})")
else:
    print("No images found.")

Predicting 4 images from 'data/negative'...

data/negative/Patient_2_Image_4.png | class = 0 | prob = 0.1168
data/negative/Patient_2_Image_2.png | class = 0 | prob = 0.1920
data/negative/Patient_2_Image_3.png | class = 0 | prob = 0.0786
data/negative/Patient_2_Image_1.png | class = 0 | prob = 0.2124

Overall patient class = 0 (ratio = 0.00)
